1. Step -1- Building a Custom Decision Tree with Information Gain:

Custom Built Decision Tree:

In [37]:
import numpy as np

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        if len(np.unique(y)) == 1:
            return {"class": y[0]}

        if self.max_depth and depth >= self.max_depth:
            return {"class": np.bincount(y).argmax()}

        best_gain = -1
        best_split = None

        for feature in range(X.shape[1]):
            for threshold in np.unique(X[:, feature]):
                left = y[X[:, feature] <= threshold]
                right = y[X[:, feature] > threshold]

                gain = self._information_gain(y, left, right)

                if gain > best_gain:
                    best_gain = gain
                    best_split = (feature, threshold)

        if best_split is None:
            return {"class": np.bincount(y).argmax()}

        feature, threshold = best_split
        left_mask = X[:, feature] <= threshold
        right_mask = X[:, feature] > threshold

        return {
            "feature": feature,
            "threshold": threshold,
            "left": self._build_tree(X[left_mask], y[left_mask], depth+1),
            "right": self._build_tree(X[right_mask], y[right_mask], depth+1)
        }

    def _entropy(self, y):
        probs = np.bincount(y) / len(y)
        return -np.sum(probs * np.log2(probs + 1e-9))

    def _information_gain(self, parent, left, right):
        return self._entropy(parent) - (
            (len(left)/len(parent))*self._entropy(left) +
            (len(right)/len(parent))*self._entropy(right)
        )

    def predict(self, X):
        return [self._predict_one(x, self.tree) for x in X]

    def _predict_one(self, x, tree):
        if "class" in tree:
            return tree["class"]
        if x[tree["feature"]] <= tree["threshold"]:
            return self._predict_one(x, tree["left"])
        else:
            return self._predict_one(x, tree["right"])


Step -2- Load and Split the Iris Datasets:

In [38]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Step -3- Train and Evaluate a Custom Decision Tree:

In [39]:
from sklearn.metrics import accuracy_score

custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

y_pred_custom = custom_tree.predict(X_test)
print("Custom Tree Accuracy:", accuracy_score(y_test, y_pred_custom))


Custom Tree Accuracy: 1.0


Step -4- Train and Evaluate a Scikit Learn Decision Tree:

In [40]:
from sklearn.tree import DecisionTreeClassifier

sk_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sk_tree.fit(X_train, y_train)

y_pred_sk = sk_tree.predict(X_test)
print("Sklearn Tree Accuracy:", accuracy_score(y_test, y_pred_sk))


Sklearn Tree Accuracy: 1.0


Step -5- Result Comparison:

In [41]:
print("Custom Tree:", accuracy_score(y_test, y_pred_custom))
print("Sklearn Tree:", accuracy_score(y_test, y_pred_sk))

Custom Tree: 1.0
Sklearn Tree: 1.0


Exercise - Ensemble Methods and Hyperparameter Tuning.

1. Implement Classification Models:

In [42]:
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

dt = DecisionTreeClassifier()
rf = RandomForestClassifier()

dt.fit(X_train, y_train)
rf.fit(X_train, y_train)

print("Decision Tree F1:", f1_score(y_test, dt.predict(X_test), average="macro"))
print("Random Forest F1:", f1_score(y_test, rf.predict(X_test), average="macro"))


Decision Tree F1: 0.9198950455317179
Random Forest F1: 1.0


2. Hyperparameter Tuning:

In [43]:
from sklearn.model_selection import GridSearchCV

params = {
    "n_estimators": [50, 100],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5]
}

grid = GridSearchCV(RandomForestClassifier(), params, cv=5, scoring="f1_macro")
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)


Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}


3. Implement Regression Model:

In [44]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

dt_reg = DecisionTreeRegressor()
rf_reg = RandomForestRegressor()

dt_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)


RandomForestRegressor()